<a href="https://colab.research.google.com/github/xxshenanigans/AIB_Project5/blob/main/project6_recommend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

# 데이터셋 불러오기

- 학습자 성취수준 데이터셋 (정오답표 / 문항 정보 / 학습자 정보) <br/>
 초등학교 1학년부터 중학교 3학년까지 각 학년별 수학 교과의 작은 개념 단위(중단원)로 시행되는 평가(시험)에 대한 정오답표, 문항 정보, 학습자 정보 데이터셋
- 수학 지식체계 데이터셋 <br/>
 수학 교육 과정을 토대로 단일 수직적 관계를 맺는 후수단위개념과 선수단위개념을 구조화 한 분류체계

Training Set 9학년 정오답표
 - `learnerID` : 학습자 ID
 - `learnerProfile` : 성별;학교급;학년
 - `testID` : 평가 ID 
 - `assessmentItemID` : 평가 문항 ID
 - `answerCode` : 정오답 (`1`:정답, `0`:오답)
 - `Timestamp` : 데이터가 생성/수정 된 시간

In [4]:
df_ans = pd.read_csv('/content/drive/MyDrive/AIB/project/training.csv')

In [5]:
df_ans = df_ans.sort_values(by=['learnerID', 'testID', 'assessmentItemID']).reset_index().drop(columns='index')
df_ans

,learnerID,learnerProfile,testID,assessmentItemID,answerCode,Timestamp
0,A090000001,M;S01;9,A090000015,A090015001,1,2020-03-04 01:56:09
1,A090000001,M;S01;9,A090000015,A090015002,0,2020-03-04 01:56:12
2,A090000001,M;S01;9,A090000015,A090015003,0,2020-03-04 01:56:14
3,A090000001,M;S01;9,A090000015,A090015004,1,2020-03-04 01:56:16
4,A090000001,M;S01;9,A090000015,A090015005,0,2020-03-04 01:56:18
...,...,...,...,...,...,...
86070,A090000774,F;S01;9,A090000055,A090055002,0,2020-10-23 02:48:49
86071,A090000774,F;S01;9,A090000055,A090055003,0,2020-10-23 02:48:50
86072,A090000774,F;S01;9,A090000055,A090055004,1,2020-10-23 02:48:52
86073,A090000774,F;S01;9,A090000055,A090055005,0,2020-10-23 02:48:54


Training Set 9학년 문항 정보
 - `difficultyLevel` : 문항 난이도
 - `discriminationLevel` : 문항 변별도
 - `guessLevel` : 문항 추측도
 - `knowledgeTag` : 수학 지식체계에 따른 개념 단위 지식태그 

In [6]:
df_test = pd.read_csv('/content/drive/MyDrive/AIB/project/test.csv')

In [7]:
df_test = df_test.sort_values(by=['testID','assessmentItemID']).reset_index().drop(columns=['index'])
df_test

,testID,assessmentItemID,difficultyLevel,discriminationLevel,guessLevel,knowledgeTag,Timestamp
0,A090000001,A090001001,0.602270,2.246116,1.536600e-01,4697,2021-01-08 06:08:25.336946
1,A090000001,A090001002,0.123169,3.074208,1.303677e-01,4697,2021-01-08 06:08:25.336946
2,A090000001,A090001003,-0.071170,15.701788,2.375048e-01,10174,2021-01-08 06:08:25.336946
3,A090000001,A090001004,0.523048,2.852689,2.285441e-01,78,2021-01-08 06:08:25.336946
4,A090000001,A090001005,-0.440225,1.987888,1.077960e-05,78,2021-01-08 06:08:25.336946
...,...,...,...,...,...,...,...
510,A090000074,A090074002,0.012002,54.381941,2.930000e-08,4243,2021-01-08 06:08:25.336946
511,A090000074,A090074003,-0.725795,0.416779,1.159106e-02,4243,2021-01-08 06:08:25.336946
512,A090000074,A090074004,0.450923,4.251689,0.000000e+00,10196,2021-01-08 06:08:25.336946
513,A090000074,A090074005,2.126145,0.240034,3.483607e-02,2648,2021-01-08 06:08:25.336946


Training Set 9학년 학습자 정보
 - `theta` : 학습자의 이해력 수준
 - `realScore` : 학습자의 진점수 

In [8]:
df_user = pd.read_csv('/content/drive/MyDrive/AIB/project/learner.csv')
df_user = df_user.sort_values(by=['learnerID', 'testID']).reset_index().drop(columns=['index'])
df_user

,learnerID,learnerProfile,testID,theta,realScore,Timestamp
0,A090000001,M;S01;9,A090000015,0.188240,0.379781,2021-01-08 06:08:25.336946
1,A090000002,M;S01;9,A090000001,1.222495,0.896270,2021-01-08 06:08:25.336946
2,A090000002,M;S01;9,A090000003,1.225620,0.826820,2021-01-08 06:08:25.336946
3,A090000002,M;S01;9,A090000005,0.837591,0.662521,2021-01-08 06:08:25.336946
4,A090000002,M;S01;9,A090000007,0.691982,0.851483,2021-01-08 06:08:25.336946
...,...,...,...,...,...,...
12286,A090000773,F;S01;9,A090000048,-0.667474,0.228380,2021-01-08 06:08:25.336946
12287,A090000773,F;S01;9,A090000055,0.582190,0.599527,2021-01-08 06:08:25.336946
12288,A090000774,F;S01;9,A090000042,0.075420,0.282065,2021-01-08 06:08:25.336946
12289,A090000774,F;S01;9,A090000049,-0.512857,0.204075,2021-01-08 06:08:25.336946


수학 지식체계
- `fromConcept` : 후수단위개념
- `toConcept` : 선수단위개념

In [9]:
data_tree = pd.read_json('/content/drive/MyDrive/AIB/project/수학 지식체계 데이터 세트_210611.json')
df_tree = pd.DataFrame(data_tree).T
df_tree

,fromConcept,toConcept
0,"{'id': 3249, 'name': '거듭제곱', 'semester': '고등-수...","{'id': 1442, 'name': '거듭제곱', 'semester': '중등-중..."
1,"{'id': 3, 'name': '지수가 자연수일 때의 지수법칙', 'semeste...","{'id': 4659, 'name': '지수법칙 (2) - 지수의 곱', 'seme..."
2,"{'id': 3, 'name': '지수가 자연수일 때의 지수법칙', 'semeste...","{'id': 4660, 'name': '지수법칙 (3) - 지수의 차', 'seme..."
3,"{'id': 3, 'name': '지수가 자연수일 때의 지수법칙', 'semeste...","{'id': 4661, 'name': '지수법칙 (4) - 지수의 분배', 'sem..."
4,"{'id': 4, 'name': '거듭제곱근', 'semester': '고등-수1-...","{'id': 3249, 'name': '거듭제곱', 'semester': '고등-수..."
...,...,...
3441,"{'id': 11269, 'name': '천만 단위까지의 수 알아보기', 'seme...","{'id': 7268, 'name': '다섯 자릿수', 'semester': '초등..."
3442,"{'id': 11270, 'name': '천억 단위까지의 수 알아보기', 'seme...","{'id': 11269, 'name': '천만 단위까지의 수 알아보기', 'seme..."
3443,"{'id': 11271, 'name': '천조 단위까지의 수 알아보기', 'seme...","{'id': 11270, 'name': '천억 단위까지의 수 알아보기', 'seme..."
3444,"{'id': 8481, 'name': '각도의 합과 차', 'semester': '...","{'id': 7310, 'name': '받아 올림이 세 번 있는 $(세자릿수)+(세..."


#추천 알고리즘

In [10]:
# 학습자 ID 입력 

learnerID = input()

A090000025


In [11]:
# 학습자 정보 

learner = df_user[df_user['learnerID']==learnerID].reset_index().drop(columns=['index', 'Timestamp'])
print(learner.shape)
learner.head()

(50, 5)


,learnerID,learnerProfile,testID,theta,realScore
0,A090000025,F;S01;9,A090000001,1.222495,0.896270
1,A090000025,F;S01;9,A090000002,-0.817804,0.244996
2,A090000025,F;S01;9,A090000003,1.036362,0.788194
3,A090000025,F;S01;9,A090000005,-0.737875,0.187517
4,A090000025,F;S01;9,A090000007,-0.228735,0.389160


In [12]:
# 학습자가 푼 평가 ID(testID) 저장 

ids = []
for i in learner['testID']:
    ids.append(i)

In [13]:
# 학습자가 푼 평가의 문항 저장 

idxs = []
for i in range(len(df_test)):
    if df_test['testID'][i] in ids:
        idxs.append(i)

In [14]:
# 문항 정보 

test = df_test.iloc[idxs].reset_index().drop(columns=['index', 'Timestamp'])
print(test.shape)
test.head()

(347, 6)


,testID,assessmentItemID,difficultyLevel,discriminationLevel,guessLevel,knowledgeTag
0,A090000001,A090001001,0.602270,2.246116,0.153660,4697
1,A090000001,A090001002,0.123169,3.074208,0.130368,4697
2,A090000001,A090001003,-0.071170,15.701788,0.237505,10174
3,A090000001,A090001004,0.523048,2.852689,0.228544,78
4,A090000001,A090001005,-0.440225,1.987888,0.000011,78


In [15]:
# 정오답표 

answer = df_ans[df_ans['learnerID']==learnerID].reset_index().drop(columns=['index', 'Timestamp'])
print(answer.shape)
answer.head()

(347, 5)


,learnerID,learnerProfile,testID,assessmentItemID,answerCode
0,A090000025,F;S01;9,A090000001,A090001001,1
1,A090000025,F;S01;9,A090000001,A090001002,1
2,A090000025,F;S01;9,A090000001,A090001003,1
3,A090000025,F;S01;9,A090000001,A090001004,1
4,A090000025,F;S01;9,A090000001,A090001005,1


In [16]:
# 문항 정보와 정오답표 데이터프레임 결합 

ans_test = pd.merge(answer, test)
ans_test

,learnerID,learnerProfile,testID,assessmentItemID,answerCode,difficultyLevel,discriminationLevel,guessLevel,knowledgeTag
0,A090000025,F;S01;9,A090000001,A090001001,1,0.602270,2.246116,1.536600e-01,4697
1,A090000025,F;S01;9,A090000001,A090001002,1,0.123169,3.074208,1.303677e-01,4697
2,A090000025,F;S01;9,A090000001,A090001003,1,-0.071170,15.701788,2.375048e-01,10174
3,A090000025,F;S01;9,A090000001,A090001004,1,0.523048,2.852689,2.285441e-01,78
4,A090000025,F;S01;9,A090000001,A090001005,1,-0.440225,1.987888,1.077960e-05,78
...,...,...,...,...,...,...,...,...,...
342,A090000025,F;S01;9,A090000074,A090074002,1,0.012002,54.381941,2.930000e-08,4243
343,A090000025,F;S01;9,A090000074,A090074003,1,-0.725795,0.416779,1.159106e-02,4243
344,A090000025,F;S01;9,A090000074,A090074004,1,0.450923,4.251689,0.000000e+00,10196
345,A090000025,F;S01;9,A090000074,A090074005,0,2.126145,0.240034,3.483607e-02,2648


In [17]:
# 학습자 정보 데이터프레임에 평가별 정답률 avr 추가  
# 정답률 = (정답 수/전체 문항 수)*100 

avr = list(answer['answerCode'].groupby(answer['testID']).mean())
learner['avr'] = avr 

In [18]:
learner

,learnerID,learnerProfile,testID,theta,realScore,avr
0,A090000025,F;S01;9,A090000001,1.222495,0.896270,1.000000
1,A090000025,F;S01;9,A090000002,-0.817804,0.244996,0.250000
2,A090000025,F;S01;9,A090000003,1.036362,0.788194,0.875000
3,A090000025,F;S01;9,A090000005,-0.737875,0.187517,0.166667
4,A090000025,F;S01;9,A090000007,-0.228735,0.389160,0.333333
5,A090000025,F;S01;9,A090000009,0.161151,0.423178,0.500000
6,A090000025,F;S01;9,A090000011,1.527275,0.937276,1.000000
7,A090000025,F;S01;9,A090000013,1.035301,0.837741,0.875000
8,A090000025,F;S01;9,A090000014,1.027084,0.731992,0.833333
9,A090000025,F;S01;9,A090000015,1.191273,0.930928,1.000000


In [19]:
# 학습자의 이해도(theta)와 정답률(avr)이 모두 낮은 평가에 대하여 오답(0)인 문항을 tag에 저장 

thetas = list(learner.sort_values(by=['theta'])[:10]['testID'])
avrs = list(learner.sort_values(by=['avr'])[:10]['testID'])
tests = [i for i in avrs if i in thetas]

In [20]:
tests, len(tests)

(['A090000069',
  'A090000005',
  'A090000042',
  'A090000066',
  'A090000002',
  'A090000070',
  'A090000032',
  'A090000007'],
 8)

In [21]:
idxs = [i for i in range(len(ans_test)) if ans_test['testID'][i] in tests]

tag = ans_test.iloc[idxs]
tag = tag[ans_test['answerCode']==0]
tag

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,learnerID,learnerProfile,testID,assessmentItemID,answerCode,difficultyLevel,discriminationLevel,guessLevel,knowledgeTag
9,A090000025,F;S01;9,A090000002,A090002002,0,0.110949,1.293208,9.000000e-10,78
10,A090000025,F;S01;9,A090000002,A090002003,0,0.028607,5.642656,0.000000e+00,78
11,A090000025,F;S01;9,A090000002,A090002004,0,3.742842,0.646146,2.300000e-09,4697
12,A090000025,F;S01;9,A090000002,A090002005,0,0.633748,38.230238,2.116269e-01,4699
13,A090000025,F;S01;9,A090000002,A090002006,0,-0.700014,19.988304,2.352328e-01,78
15,A090000025,F;S01;9,A090000002,A090002008,0,-0.027761,4.690399,0.000000e+00,4697
24,A090000025,F;S01;9,A090000005,A090005001,0,1.078435,1.480927,8.657004e-02,4723
25,A090000025,F;S01;9,A090000005,A090005002,0,0.625474,4.024864,2.153106e-01,4723
26,A090000025,F;S01;9,A090000005,A090005003,0,0.085628,1.551685,6.333163e-04,4699
27,A090000025,F;S01;9,A090000005,A090005004,0,1.003348,3.773352,5.495616e-02,4709


In [22]:
# tag 문항에 대하여 지식태그 count : 오답이 가장 많은 개념(취약점)을 나타낸다. 

tag_sub = pd.DataFrame(tag['knowledgeTag'].value_counts())
tag_sub

,knowledgeTag
2612,6
9924,4
4697,3
10607,3
9929,3
78,3
10624,3
4723,3
4699,2
2644,2


In [23]:
# 오답이 1개인 학습태그는 제외하고 출력  

if tag_sub['knowledgeTag'].iloc[0] == 1:
    tags = list(tag_sub.index)
else:
    tags = list(tag_sub[tag_sub['knowledgeTag'] > 1].index)
print(tags)

[2612, 9924, 4697, 10607, 9929, 78, 10624, 4723, 4699, 2644, 4725]


##함수

In [30]:
def inputTag():

    """
    입력된 학습자ID의 취약점을 진단하여 해당 단원의 학습태그를 출력하는 함수 
    """

    # 학습자ID 입력
    learnerID = input()

    # 학습자 데이터셋(df_user)에서 학습자ID 데이터 추출 >> 이해도(theta) 포함
    learner = df_user[df_user['learnerID']==learnerID].reset_index().drop(columns=['index', 'Timestamp'])

    # 평가ID 추출 
    ids = [i for i in learner['testID']]

    # 평가 데이터셋(df_test)에서 평가ID 데이터 추출 >> 난이도(difficultyLevel), 변별도(discriminationLevel), 학습태그(knowledgeTag) 포함 
    idxs = [i for i in range(len(df_test)) if df_test['testID'][i] in ids]
    test = df_test.iloc[idxs].reset_index().drop(columns=['index', 'Timestamp'])

    # 정오답표 데이터셋(df_ans)에서 학습자 ID 데이터 추출 >> 정오답(answerCode) 포함 / 정답률 계산 
    answer = df_ans[df_ans['learnerID']==learnerID].reset_index().drop(columns=['index', 'Timestamp'])

    # answer, test 결합 (평가ID, 문항ID 중복)
    ans_test = pd.merge(answer, test)

    # 각 평가에 따른 정답률을 구하여 학습자 데이터셋에 추가 
    avr = list(answer['answerCode'].groupby(answer['testID']).mean())
    learner['avr'] = avr

    # 학습자의 이해도(theta)와 정답률(avr)이 모두 낮은 평가에 대하여 오답(0)인 문항 추출  
    thetas = list(learner.sort_values(by=['theta'])[:10]['testID'])
    avrs = list(learner.sort_values(by=['avr'])[:10]['testID'])

    tests = [i for i in avrs if i in thetas]
    idxs = [i for i in range(len(ans_test)) if ans_test['testID'][i] in tests]
    
    tag = ans_test.iloc[idxs]
    tag = tag[tag['answerCode']==0]
    
    # 학습태그 출력 (오답이 1개인 학습태그는 제외)
    tag_sub = pd.DataFrame(tag['knowledgeTag'].value_counts())

    if tag_sub['knowledgeTag'].iloc[0] == 1:
        tags = list(tag_sub.index)
    else:
        tags = list(tag_sub[tag_sub['knowledgeTag'] > 1].index)

    return tags

In [31]:
inputTag()

A090000025


[2612, 9924, 4697, 10607, 9929, 78, 10624, 4723, 4699, 2644, 4725]

In [32]:
def outputTag():

    """
    inputTag()의 학습태그를 입력받아 취약한 순서대로 단원 정보를 출력하는 함수
    """
    
    tags = inputTag()
#    fromc = []
#    toc = []

#    for i in range(len(df_tree)):
#        if df_tree['fromConcept'][i]['id'] in tags:
#            fromc.append(df_tree['fromConcept'][i])
#            toc.append(df_tree['toConcept'][i])
    
    idx = []
    id = []
    name = []
    semester = []
    achievement = []
    chapter = []
    description = []
    for j in tags:
        for i in range(len(df_tree)):
            if j == df_tree['fromConcept'][i]['id']:
                idx.append('취약단원')
                idx.append('선수단원')
                id.append(df_tree['fromConcept'][i]['id'])
                id.append(df_tree['toConcept'][i]['id'])
                name.append(df_tree['fromConcept'][i]['name'])
                name.append(df_tree['toConcept'][i]['name'])
                semester.append(df_tree['fromConcept'][i]['semester'])
                semester.append(df_tree['toConcept'][i]['semester'])
                achievement.append(df_tree['fromConcept'][i]['achievement']['name'])
                achievement.append(df_tree['toConcept'][i]['achievement']['name'])
                chapter.append(df_tree['fromConcept'][i]['chapter']['name'])
                chapter.append(df_tree['toConcept'][i]['chapter']['name'])
                description.append(df_tree['fromConcept'][i]['description'])
                description.append(df_tree['toConcept'][i]['description'])

    res = pd.DataFrame({'idx':idx,
                        'id':id,
                        'name':name,
                        'semester':semester,
                        'achievement':achievement,
                        'chapter':chapter,
                        'description':description})

    return res

In [33]:
res = outputTag()
res

A090000025


,idx,id,name,semester,achievement,chapter,description
0,취약단원,2612,원에 내접하는 사각형의 성질,중등-중3-2학기,원주각의 성질을 이해한다.,원의 성질 > 원주각 > 원주각과 그 성질,(1) 원에 내접하는 사각형에서 한 쌍의 대각의 크기의 합은 $180^{\circ}...
1,선수단원,2610,원주각의 성질,중등-중3-2학기,원주각의 성질을 이해한다.,원의 성질 > 원주각 > 원주각과 그 성질,(1) 원에서 한 호에 대한 원주각의 크기는 모두 같다. \n→ $\angle AP...
2,취약단원,9924,평균,중등-중3-2학기,"중앙값, 최빈값, 평균의 의미를 이해하고 이를 구할 수 있다.",통계 > 대푯값과 산포도 > 대푯값,평균 : 전체 변량의 총합을 변량의 개수로 나눈 값\n→ $(평균)=\frac{(변...
3,선수단원,2643,대푯값,중등-중3-2학기,"중앙값, 최빈값, 평균의 의미를 이해하고, 이를 구할 수 있다.",통계 > 대푯값과 산포도 > 대푯값,대푯값 : 자료 전체의 중심적인 경향이나 특징을 대표적인 하나의 수로 나타낸 값
4,취약단원,4697,제곱근의 표현,중등-중3-1학기,"제곱근의 뜻을 알고, 그 성질을 이해한다.",제곱근과 실수 > 제곱근과 실수 > 제곱근의 뜻,(1) 제곱근은 기호 $\sqrt{\placeholder{}}$를 사용하여 나타내는...
5,선수단원,4604,유리수,중등-중2-1학기,"순환소수의 뜻을 알고, 유리수와 순환소수의 관계를 이해한다.",유리수와 소수 > 유리수와 소수 > 순환소수,"a, b 가 정수이고 $be0$ 일때, 분수 $\frac{a}{b}$ 의 꼴로 나타..."
6,취약단원,10607,이차방정식의 근의 개수,중등-중3-1학기,이차방정식을 활용하여 문제를 해결할 수 있다.,이차방정식 > 이차방정식 > 근의 공식을 이용한 이차방정식의 풀이,이차방정식 $ax^2+bx+c=0$의 근은 $x=\frac{-b\pm\sqrt[]{...
7,선수단원,1011,이차방정식의 근의 공식; $ax^2+bx+c=0$,중등-중3-1학기,이차방정식을 풀 수 있다.,이차방정식 > 이차방정식 > 근의공식을 이용한 이차방정식의 풀이,다음과 같이 이차방정식의 근을 구하는 공식을 이차방정식의 근의 공식이라 한다. \n...
8,취약단원,78,제곱근,중등-중3-1학기,"제곱근의 뜻을 알고, 그 성질을 이해한다.",제곱근과 실수 > 제곱근과 실수 > 제곱근의 뜻,"어떤 수 $x$를 제곱하여 $a$가 될 때, 즉\n$x^2=a$\n일 때, $x$를..."
9,선수단원,4604,유리수,중등-중2-1학기,"순환소수의 뜻을 알고, 유리수와 순환소수의 관계를 이해한다.",유리수와 소수 > 유리수와 소수 > 순환소수,"a, b 가 정수이고 $be0$ 일때, 분수 $\frac{a}{b}$ 의 꼴로 나타..."


In [34]:
# description의 LaTeX 출력 (Colab으로 불러오면 더 잘 읽는다.)

from IPython.display import display, Markdown, Latex

for i in res['description']:
    i = i.replace('\\n', '\n')
    display(Markdown(i))
    print('')

(1) 원에 내접하는 사각형에서 한 쌍의 대각의 크기의 합은 $180^{\circ}$이다. 
→ $\angle A+\angle C=180^{\circ}$, $\angle B+\angle D=180^{\circ}$

(2) 원에 내접하는 사각형에서 한 외각의 크기는 그와 이웃한 내각의 대각의 크기와 같다. 
→ $\angle DCE=\angle A$

(1) 원에서 한 호에 대한 원주각의 크기는 모두 같다. 
→ $\angle AP_1B=\angle AP_2B=\angle AP_3B$

(2) 반원에 대한 원주각의 크기는$90^{\circ}$이다. 
→ $\overline{AB}$가 원의 지름이면 $\angle APB=90^{\circ}$

평균 : 전체 변량의 총합을 변량의 개수로 나눈 값
→ $(평균)=\frac{(변량의 총합)}{(변량의 개수)}$

대푯값 : 자료 전체의 중심적인 경향이나 특징을 대표적인 하나의 수로 나타낸 값

(1) 제곱근은 기호 $\sqrt{\placeholder{}}$를 사용하여 나타내는데 이것을 근호라 하고, '제곱근' 또는 '루트(root)'라 읽는다. 

(2) 양수 a의 제곱근 중에서 양수인 것을 양의 제곱근, 음수인 것을 음의 제곱근이라 하고 다음과 같이 나타낸다.
양의 제곱근 : $\sqrt[]{a}$
음의 제곱근 : $-\sqrt[]{a}$

...

(1) 부등식의 양변에 같은 수를 더하거나 양변에서 같은 수를 빼어도 부등호의 방향은 바뀌지 않는다.
=> a<b이면 a+c<b+c , a-c<b-c
(2) 부등식의 양변에 같은 양수를 곱하거나 양변을 같은 양수로 나누어도 부등호의 방향은 바뀌지 않는다. 
=> a<b, c>0이면 ac<bc, $\frac{a}{c}<\frac{b}{c}$ (부등호의 방향 그대로)
(3) 부등식의 양변에 같은 음수를 곱하거나 양변을 같은 음수로 나누면 부등호의 방향이 바뀐다.
=> a<b, c<0이면 ac>bc, $\frac{a}{c}>\frac{b}{c}$ (부등호의 방향 반대로)

(1) 양수 $a$의 제곱근 $\sqrt{a}$와 $-\sqrt{a}$는 제곱하면 $a$가 된다.
즉, $a>0$일 때, $(\sqrt{a})^2=a$,
$(-\sqrt{a})^2=a$
(2) 근호 안의 수가 어떤 수의 제곱이면 근호를 사용하지 않고 나타낼 수 있다.
즉, $a>0$일 때, $\sqrt{a^2}=a$, $\sqrt{(-a)^2}=a$

(1) 제곱근은 기호 $\sqrt{\placeholder{}}$를 사용하여 나타내는데 이것을 근호라 하고, '제곱근' 또는 '루트(root)'라 읽는다. 

(2) 양수 a의 제곱근 중에서 양수인 것을 양의 제곱근, 음수인 것을 음의 제곱근이라 하고 다음과 같이 나타낸다.
양의 제곱근 : $\sqrt[]{a}$
음의 제곱근 : $-\sqrt[]{a}$

자료의 변량을 작은 값부터 크기순으로 나열할 때, 중앙에 위치하는 값을 그 자료의 중앙값이라 한다.
(1) 변량의 개수가 홀수이면 중앙에 위치하는 값이 중앙값이다.
(2) 변량의 개수가 짝수이면 중앙에 위치하는 두 값의 평균이 중앙값이다.

대푯값 : 자료 전체의 중심적인 경향이나 특징을 대표적인 하나의 수로 나타낸 값